1. Load files from a directory
2. Find all xpaths
3. Order acc to xsd
4. Check if it is repeating
5. Get Desc from xsd

In [35]:
from lxml import etree as ElementTree
import pandas as pd

# Nahrazení namespaců, aby bylo lépe čitelné

or_namespaces = {'are': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_answer_or/v_1.0.2', 
              'dtt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_datatypes/v_1.0.2',
              'udt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/uvis_datatypes/v_1.0.1',
              'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
             }
inverse_or_namespaces = {v: k for k, v in or_namespaces.items()}
def replace_ns(path_w_ns):
    for key, name in inverse_or_namespaces.items():
        path_w_ns = path_w_ns.replace(key, name)
    path_wo_ns = path_w_ns
    path_wo_ns = path_wo_ns.replace('{','')
    path_wo_ns = path_wo_ns.replace('}',':')
    return path_wo_ns

# Elementy XML a jejich xpath cesty
def etree_iter_path(node, tag=None, path='.'):
    if tag == "*":
        tag = None
    if tag is None or node.tag == tag:
        yield node, path
    for child in node:
        _child_path = '%s/%s' % (path, child.tag)
        for child, child_path in etree_iter_path(child, tag, path=_child_path):
            yield child, child_path

# Dotažení info o XML do pd.DataFrame
def xml_paths_no_dupl(filename):
    xmldoc = ElementTree.parse(filename)
    list_data = []
    for elem, path in etree_iter_path(xmldoc.getroot()):
        elem_parent = ''
        if elem.getparent() is not None:
            elem_parent = replace_ns(elem.getparent().tag)
        attribu = elem.attrib
        attr_keys_string = ",".join(attribu.keys())
        list_data.append([replace_ns(elem.tag), elem_parent,attr_keys_string,replace_ns(path)])
    pd_data = pd.DataFrame(list_data, columns = ['Tag','Parent_Tag','Attributes','xPath'])
    pd_no_dupl = pd_data.drop_duplicates()
    return pd_no_dupl

In [10]:
import os
files_to_profile = os.listdir('./xml_examples')
files_to_profile

['00000515.xml',
 '00000744.xml',
 '00000795.xml',
 '00000914.xml',
 '00006025.xml',
 '00012661.xml',
 '00036421.xml',
 '00037494.xml',
 '00204960.xml',
 '00207225.xml']

In [38]:
complet = pd.DataFrame(columns = ['Tag','Parent_Tag','Attributes','xPath'])

for file in files_to_profile:
    filename = './xml_examples/'+file
    result = xml_paths_no_dupl(filename)
    complet = pd.concat([complet, result])

In [39]:
complet

,Tag,Parent_Tag,Attributes,xPath
0,are:Ares_odpovedi,,"odpoved_datum_cas,odpoved_pocet,odpoved_typ,vy...",.
1,are:Odpoved,are:Ares_odpovedi,,./are:Odpoved
2,dtt:Pomocne_ID,are:Odpoved,,./are:Odpoved/dtt:Pomocne_ID
3,dtt:Vysledek_hledani,are:Odpoved,,./are:Odpoved/dtt:Vysledek_hledani
4,dtt:Kod,dtt:Vysledek_hledani,,./are:Odpoved/dtt:Vysledek_hledani/dtt:Kod
5,dtt:Pocet_zaznamu,are:Odpoved,,./are:Odpoved/dtt:Pocet_zaznamu
6,dtt:Vypis_OR,are:Odpoved,,./are:Odpoved/dtt:Vypis_OR
7,dtt:Uvod,dtt:Vypis_OR,,./are:Odpoved/dtt:Vypis_OR/dtt:Uvod
8,dtt:Nadpis,dtt:Uvod,,./are:Odpoved/dtt:Vypis_OR/dtt:Uvod/dtt:Nadpis
9,dtt:Aktualizace_DB,dtt:Uvod,,./are:Odpoved/dtt:Vypis_OR/dtt:Uvod/dtt:Aktual...


In [40]:
comp_no_dup=complet.drop_duplicates()

In [41]:
"""Zapsání """
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('xml_profile.xlsx')

# Convert the dataframe to an XlsxWriter Excel object.
comp_no_dup.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()